In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import date, timedelta

In [2]:
def find_sheet_name(sheet_names):
    iip_sheet_names = list(filter(lambda x: "iip" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(filter(lambda x: "thang" in x.lower(), iip_sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    if len(sheet_names) == 43:
        return "7"

    iip_sheet_names = list(filter(lambda x: "gtcn" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    raise RuntimeError

In [3]:
en_to_vn = {
    "WHOLE INDUSTRY": "Toàn ngành công nghiệp",
    "Mining and quarying": "Khai khoáng",
    "Mining of coal and lignite": "Khai thác than cứng và than non",
    "Extraction of crude petroleum and nutural gas": "Khai thác dầu thô và khí đốt tự nhiên",
    "Mining of metal ores": "Khai thác quặng kim loại",
    "Other mining and quarrying (stone, sand and clay)": "Khai khoáng khác",
    "Mining support service activities": "Hoạt động dịch vụ hỗ trợ khai thác mỏ và quặng",
    "Manufacturing": "Công nghiệp chế biến, chế tạo",
    "Manufacture of food products": "Sản xuất, chế biến thực phẩm",
    "Manufacture of beverages": "Sản xuất đồ uống",
    "Manufacture of tobacco products": "Sản xuất sản phẩm thuốc lá",
    "Manufacture of textiles": "Dệt",
    "Manufacture of wearing apparel": "Sản xuất trang phục",
    "Manufacture of leather and related products": "Sản xuất da và các sản phẩm có liên quan",
    "Manufacture of wood and of products of wood and cork": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "Manufacture of paper and paper products": "Sản xuất giấy và sản phẩm từ giấy",
    "Printing and reproduction of recorded media": "In, sao chép bản ghi các loại",
    "Manufacture of coke and refined petroleum products": "Sản xuất than cốc, sản phẩm dầu mỏ tinh chế",
    "Manufacture of chemicals and chemical products": "Sản xuất hoá chất và sản phẩm hoá chất",
    "Manufacture of pharmaceuticals, medicinal chemical and botanical products": "Sản xuất thuốc, hoá dược và dược liệu",
    "Manufacture of rubber and plastics products": "Sản xuất sản phẩm từ cao su và plastic",
    "Manufacture of other non-metallic mineral products": "Sản xuất sản phẩm từ khoáng phi kim loại khác",
    "Manufacture of basic metals": "Sản xuất kim loại",
    "Manufacture of fabricated metal products, except machinery and equipment": "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)",
    "Manufacture of computer, electronic and optical products": "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học",
    "Manufacture of electrical equipment": "Sản xuất thiết bị điện",
    "Manufacture of machinery and equipment n.e.c": "Sản xuất máy móc, thiết bị chưa được phân vào đâu",
    "Manufacture of motor vehicles; trailers and semi-trailers": "Sản xuất xe có động cơ",
    "Manufacture of other transport equipment": "Sản xuất phương tiện vận tải khác",
    "Manufacture of furniture": "Sản xuất giường, tủ, bàn, ghế",
    "Other manufacturing": "Công nghiệp chế biến, chế tạo khác",
    "Repair and installation of machinery and equipment": "Sửa chữa, bảo dưỡng và lắp đặt máy móc, thiết bị",
    "Electricity, gas, steam and air conditioning supply": "Sản xuất và phân phối điện",
    "Water supply; sewerage, waste management and remediation activities": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "Water collection, treatment and supply": "Khai thác, xử lý và cung cấp nước",
    "Sewerage and sewer treatment activities": "Thoát nước và xử lý nước thải",
    "Waste collection, treatment and disposal activities; materials recovery": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
}

In [4]:
def get_ipp_2023(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df = df.dropna(axis=1, how="all")

    if len(df.columns) == 5:
        df.columns = [
            "Ngành",
            "YOY-Pre-Month",
            "MOM",
            "YOY",
            "YTD-YOY",
        ]
    elif len(df.columns) == 3 and int(date.split("-")[1]) == 1:
        df.columns = [
            "Ngành",
            "MOM",
            "YOY",
        ]
        df["YTD-YOY"] = df["YOY"]
        df["YOY-Pre-Month"] = np.nan
    else:
        raise RuntimeError

    assert len(df.columns) == 5

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
        .str.strip()
    )

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "Toàn ngành công nghiệp")
                    | (df[df.columns[0]] == "WHOLE INDUSTRY")
                    | (df[df.columns[0]] == "TOÀN NGÀNH CÔNG NGHIỆP")
                    | (df[df.columns[0]] == "TOÀN QUỐC")
                ].index[0]
            )
        )
    )

    if len(df["Ngành"][df["Ngành"] == "WHOLE INDUSTRY"].index) > 0:
        df["Ngành"] = df["Ngành"].map(en_to_vn)

    df = df[pd.to_numeric(df["MOM"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YOY"], errors="coerce").notnull()]
    df = df[
        pd.to_numeric(
            df["YTD-YOY"],
            errors="coerce",
        ).notnull()
    ]

    df = df[df["MOM"].notna()]
    df = df[df["YOY"].notna()]
    df = df[df["YTD-YOY"].notna()]

    df["MOM"] = (df["MOM"] - 100).astype(float)
    df["YOY"] = (df["YOY"] - 100).astype(float)
    df["YTD-YOY"] = (df["YTD-YOY"] - 100).astype(float)
    df["YOY-Pre-Month"] = (df["YOY-Pre-Month"] - 100).astype(float)

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    df["Month"] = df["Date"].dt.strftime("%Y-%m")
    df["PreMonth"] = (
        df["Date"]
        .apply(
            lambda x: (
                x.replace(month=x.month - 1, day=1)
                if x.month > 1
                else x.replace(year=x.year - 1, month=12, day=1)
            )
        )
        .dt.strftime("%Y-%m")
    )

    # df["PreYearMonth"] = (
    #     df["Date"]
    #     .apply(lambda x: x.replace(year=x.year - 1, day=1))
    #     .dt.strftime("%Y-%m")
    # )

    df = df.replace(
        {
            "rác thải, nước thải": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải"
        }
    )

    return df


xl = pd.ExcelFile("../datas/2024-06-29-02.Bieu-T6.2024.xlsx")
get_ipp_2023(xl, find_sheet_name(xl.sheet_names), "2024-06-29").head(n=15)

,Ngành,YOY-Pre-Month,MOM,YOY,YTD-YOY,Date,Month,PreMonth
7,Toàn ngành công nghiệp,10.03,0.67,10.92,7.72,2024-06-29,2024-06,2024-05
8,Khai khoáng,-8.30,-2.64,-7.68,-5.47,2024-06-29,2024-06,2024-05
9,Khai thác than cứng và than non,1.10,-7.46,1.27,0.03,2024-06-29,2024-06,2024-05
10,Khai thác dầu thô và khí đốt tự nhiên,-16.83,-1.37,-15.27,-11.71,2024-06-29,2024-06,2024-05
11,Khai thác quặng kim loại,40.11,-3.95,1.05,16.70,2024-06-29,2024-06,2024-05
12,Khai khoáng khác,5.61,-4.91,-0.85,2.22,2024-06-29,2024-06,2024-05
13,Hoạt động dịch vụ hỗ trợ khai thác mỏ và quặng,2.67,14.74,20.81,17.41,2024-06-29,2024-06,2024-05
14,"Công nghiệp chế biến, chế tạo",11.88,0.98,12.62,8.45,2024-06-29,2024-06,2024-05
15,"Sản xuất, chế biến thực phẩm",7.47,4.03,10.36,6.54,2024-06-29,2024-06,2024-05
16,Sản xuất đồ uống,5.45,-0.20,1.67,0.46,2024-06-29,2024-06,2024-05


In [5]:
import os
import contextlib

dfs_iip_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    with contextlib.suppress(Exception):
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        xl = pd.ExcelFile(file_path)

        df_iip = get_ipp_2023(
            xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}"
        )

        dfs_iip_origin.append(df_iip)

In [6]:
dfs_iip = pd.concat(dfs_iip_origin, ignore_index=True, axis=0)
dfs_iip["Date"] = pd.to_datetime(dfs_iip["Date"], dayfirst=False)
dfs_iip

,Ngành,YOY-Pre-Month,MOM,YOY,YTD-YOY,Date,Month,PreMonth
0,Toàn ngành công nghiệp,8.380000,-0.190000,10.840000,8.640000,2024-09-29,2024-09,2024-08
1,Khai khoáng,-6.100000,-7.300000,-5.900000,-6.450000,2024-09-29,2024-09,2024-08
2,Khai thác than cứng và than non,-13.880000,-9.250000,-10.030000,-4.160000,2024-09-29,2024-09,2024-08
3,Khai thác dầu thô và khí đốt tự nhiên,-6.230000,-7.310000,-7.820000,-11.500000,2024-09-29,2024-09,2024-08
4,Khai thác quặng kim loại,25.230000,-12.730000,13.200000,16.970000,2024-09-29,2024-09,2024-08
...,...,...,...,...,...,...,...,...
4232,"Sản xuất giường, tủ, bàn, ghế",-2.205523,-21.023681,28.980919,9.474830,2014-02-28,2014-02,2014-01
4233,Sản xuất và phân phối điện,4.629041,-7.278234,13.088683,8.535242,2014-02-28,2014-02,2014-01
4234,"và xử lý rác thải, nước thải",4.442353,0.587840,7.132010,5.774026,2014-02-28,2014-02,2014-01
4235,"Khai thác, xử lý và cung cấp nước",5.025523,-0.469821,6.713222,5.860660,2014-02-28,2014-02,2014-01


In [7]:
convert_nganh = {
    "Toàn ngành công nghiệp": "Toàn ngành công nghiệp",
    "Khai khoáng": "Khai khoáng",
    "Khai thác than cứng và than non": "Khai thác than cứng và than non",
    "Khai thác dầu thô và khí đốt tự nhiên": "Khai thác dầu thô và khí đốt tự nhiên",
    "Khai thác quặng kim loại": "Khai thác quặng kim loại",
    "Khai khoáng khác": "Khai khoáng khác",
    "Hoạt động dịch vụ hỗ trợ khai thác mỏ và quặng": "Hoạt động dịch vụ hỗ trợ khai thác mỏ và quặng",
    "Công nghiệp chế biến, chế tạo": "Công nghiệp chế biến, chế tạo",
    "Sản xuất, chế biến thực phẩm": "Sản xuất, chế biến thực phẩm",
    "Sản xuất đồ uống": "Sản xuất đồ uống",
    "Sản xuất sản phẩm thuốc lá": "Sản xuất sản phẩm thuốc lá",
    "Dệt": "Dệt",
    "Sản xuất trang phục": "Sản xuất trang phục",
    "Sản xuất da và các sản phẩm có liên quan": "Sản xuất da và các sản phẩm có liên quan",
    "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "Sản xuất giấy và sản phẩm từ giấy": "Sản xuất giấy và sản phẩm từ giấy",
    "In, sao chép bản ghi các loại": "In, sao chép bản ghi các loại",
    "Sản xuất than cốc, sản phẩm dầu mỏ tinh chế": "Sản xuất than cốc, sản phẩm dầu mỏ tinh chế",
    "Sản xuất hoá chất và sản phẩm hoá chất": "Sản xuất hoá chất và sản phẩm hoá chất",
    "Sản xuất thuốc, hoá dược và dược liệu": "Sản xuất thuốc, hoá dược và dược liệu",
    "Sản xuất sản phẩm từ cao su và plastic": "Sản xuất sản phẩm từ cao su và plastic",
    "Sản xuất sản phẩm từ khoáng phi kim loại khác": "Sản xuất sản phẩm từ khoáng phi kim loại khác",
    "Sản xuất kim loại": "Sản xuất kim loại",
    "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)": "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)",
    "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học": "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học",
    "Sản xuất thiết bị điện": "Sản xuất thiết bị điện",
    "Sản xuất máy móc, thiết bị chưa được phân vào đâu": "Sản xuất máy móc, thiết bị chưa được phân vào đâu",
    "Sản xuất xe có động cơ": "Sản xuất xe có động cơ",
    "Sản xuất phương tiện vận tải khác": "Sản xuất phương tiện vận tải khác",
    "Sản xuất giường, tủ, bàn, ghế": "Sản xuất giường, tủ, bàn, ghế",
    "Công nghiệp chế biến, chế tạo khác": "Công nghiệp chế biến, chế tạo khác",
    "Sửa chữa, bảo dưỡng và lắp đặt máy móc, thiết bị": "Sửa chữa, bảo dưỡng và lắp đặt máy móc, thiết bị",
    "Sản xuất và phân phối điện": "Sản xuất và phân phối điện",
    "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "Khai thác, xử lý và cung cấp nước": "Khai thác, xử lý và cung cấp nước",
    "Thoát nước và xử lý nước thải": "Thoát nước và xử lý nước thải",
    "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
    "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn, ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "Sửa chữa, bảo dưỡng và lắp đặt máy móc và thiết bị": "Sửa chữa, bảo dưỡng và lắp đặt máy móc, thiết bị",
    "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí": "Sản xuất và phân phối điện",
    "rơm, rạ và vật liệu tết bện": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "(trừ máy móc, thiết bị)": "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)",
    "và sản phẩm quang học": "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học",
    "bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện": "Chế biến gỗ và sản xuất sản phẩm từ gỗ, tre, nứa (trừ giường, tủ, bàn ghế); sản xuất sản phẩm từ rơm, rạ và vật liệu tết bện",
    "tái chế phế liệu": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
    "Sản xuất chế biến thực phẩm": "Sản xuất, chế biến thực phẩm",
    "Sản xuất thuốc lá": "Sản xuất sản phẩm thuốc lá",
    "Hoạt độg thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
    "TOÀN NGÀNH CÔNG NGHIỆP": "Toàn ngành công nghiệp",
    "và xử lý rác thải, nước thải": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "HĐ thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu": "Hoạt động thu gom, xử lý và tiêu huỷ rác thải; tái chế phế liệu",
    "Sản xuất sản phẩm điện tử, máy vi tưnh và sản phẩm quang học": "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học",
    "Công nghiệp chế biến , chế tạo": "Công nghiệp chế biến, chế tạo",
    "TOÀN QUỐC": "Toàn ngành công nghiệp",
    "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hoà không khí": "Sản xuất và phân phối điện",
}

for pl in dfs_iip["Ngành"].unique().tolist():
    assert pl in convert_nganh.keys(), pl

dfs_iip["Ngành"] = dfs_iip["Ngành"].map(convert_nganh)
dfs_iip

,Ngành,YOY-Pre-Month,MOM,YOY,YTD-YOY,Date,Month,PreMonth
0,Toàn ngành công nghiệp,8.380000,-0.190000,10.840000,8.640000,2024-09-29,2024-09,2024-08
1,Khai khoáng,-6.100000,-7.300000,-5.900000,-6.450000,2024-09-29,2024-09,2024-08
2,Khai thác than cứng và than non,-13.880000,-9.250000,-10.030000,-4.160000,2024-09-29,2024-09,2024-08
3,Khai thác dầu thô và khí đốt tự nhiên,-6.230000,-7.310000,-7.820000,-11.500000,2024-09-29,2024-09,2024-08
4,Khai thác quặng kim loại,25.230000,-12.730000,13.200000,16.970000,2024-09-29,2024-09,2024-08
...,...,...,...,...,...,...,...,...
4232,"Sản xuất giường, tủ, bàn, ghế",-2.205523,-21.023681,28.980919,9.474830,2014-02-28,2014-02,2014-01
4233,Sản xuất và phân phối điện,4.629041,-7.278234,13.088683,8.535242,2014-02-28,2014-02,2014-01
4234,Cung cấp nước; hoạt động quản lý và xử lý rác ...,4.442353,0.587840,7.132010,5.774026,2014-02-28,2014-02,2014-01
4235,"Khai thác, xử lý và cung cấp nước",5.025523,-0.469821,6.713222,5.860660,2014-02-28,2014-02,2014-01


In [8]:
merge_df = pd.merge(
    left=dfs_iip,
    right=dfs_iip,
    left_on=["Ngành", "Month"],
    right_on=["Ngành", "PreMonth"],
    how="left",
)
merge_df["Merged_YOY"] = np.where(
    merge_df["YOY-Pre-Month_y"].isna(), merge_df["YOY_x"], merge_df["YOY-Pre-Month_y"]
)
merge_df = merge_df.drop(
    columns=[
        "YOY-Pre-Month_y",
        "MOM_y",
        "YOY_y",
        "YTD-YOY_y",
        "Date_y",
        "Month_y",
        "PreMonth_y",
    ]
)
merge_df = merge_df.rename(
    columns={
        "YOY-Pre-Month_x": "YOY-Pre-Month",
        "MOM_x": "MOM",
        "YOY_x": "E-YOY",
        "YTD-YOY_x": "YTD-YOY",
        "Date_x": "Date",
        "Month_x": "Month",
        "PreMonth_x": "PreMonth",
        "Merged_YOY": "YOY",
    }
)
merge_df[merge_df["Ngành"] == "Toàn ngành công nghiệp"]

,Ngành,YOY-Pre-Month,MOM,E-YOY,YTD-YOY,Date,Month,PreMonth,YOY
0,Toàn ngành công nghiệp,8.380000,-0.190000,10.840000,8.640000,2024-09-29,2024-09,2024-08,10.840000
37,Toàn ngành công nghiệp,11.080000,2.000000,9.500000,8.600000,2024-08-29,2024-08,2024-07,8.380000
74,Toàn ngành công nghiệp,12.360000,0.690000,11.230000,8.470000,2024-07-29,2024-07,2024-06,11.080000
111,Toàn ngành công nghiệp,10.030000,0.670000,10.920000,7.720000,2024-06-29,2024-06,2024-05,12.360000
148,Toàn ngành công nghiệp,7.390000,3.850000,8.866731,6.755527,2024-05-29,2024-05,2024-04,10.030000
...,...,...,...,...,...,...,...,...,...
4098,Toàn ngành công nghiệp,5.724757,0.490725,6.095250,5.759989,2014-06-27,2014-06,2014-05,6.039949
4126,Toàn ngành công nghiệp,5.533029,2.030000,5.880000,5.639791,2014-05-29,2014-05,2014-04,5.724757
4154,Toàn ngành công nghiệp,5.291907,1.208386,6.000000,5.439391,2014-04-29,2014-04,2014-03,5.533029
4182,Toàn ngành công nghiệp,5.364297,16.900000,4.700000,5.200000,2014-03-28,2014-03,2014-02,5.291907


In [9]:
dfs_iip = merge_df

In [10]:
dfs_iip.to_csv("../csv/iip.csv")

In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_iip[
    (
        (dfs_iip["Ngành"] == "Toàn ngành công nghiệp")
        | (dfs_iip["Ngành"] == "Khai khoáng")
        | (dfs_iip["Ngành"] == "Công nghiệp chế biến, chế tạo")
        | (dfs_iip["Ngành"] == "Sản xuất và phân phối điện")
    )
    & (dfs_iip["Date"] > "2018-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")
# fig = px.area(df_show, x="Date", y="YTD-YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

In [12]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_iip[
    (
        (dfs_iip["Ngành"] == "Sản xuất kim loại")
        | (
            dfs_iip["Ngành"]
            == "Sản xuất sản phẩm từ kim loại đúc sẵn (trừ máy móc, thiết bị)"
        )
        | (dfs_iip["Ngành"] == "Dệt")
        # | (dfs_iip["Ngành"] == "Sản xuất trang phục")
        | (
            dfs_iip["Ngành"]
            == "Sản xuất sản phẩm điện tử, máy vi tính và sản phẩm quang học"
        )
        | (dfs_iip["Ngành"] == "Sản xuất thiết bị điện")
        # | (dfs_iip["Ngành"] == "Khai thác dầu thô và khí đốt tự nhiên")
        | (dfs_iip["Ngành"] == "Sản xuất sản phẩm từ cao su và plastic")
        | (dfs_iip["Ngành"] == "Sản xuất hoá chất và sản phẩm hoá chất")
        | (dfs_iip["Ngành"] == "Sản xuất, chế biến thực phẩm")
    )
    & (dfs_iip["Date"] > "2022-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")
# fig = px.area(df_show, x="Date", y="YTD-YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()